# Overview

This notebook demonstrates how to synthetically generate mixed audio files at different volumen levels.  The output files generated are used as example mixed audio signal input to deconvolute using Independent Component Analysis (ICA).  The output simulates recording from two observers in a room where a piano is playing Saint-Saëns "The Carnival of the Animals - XIII" and a cello playing Bach's "Cello Suite no. 3 in C major".  In one recording, the cello is louder, but the piano can be heard in the background.  And vice versa for the other recording.

# Prepare input

## Download input files

Download the following open source recordings.

- [The Carnival of the Animals - XIII. The Swan (Solo piano version)](https://musopen.org/music/1454-the-carnival-of-the-animals/#recordings)
- [Cello Suite no. 3 in C, BWV 1009 - 3. Courante](https://musopen.org/music/4001-cello-suite-no-3-in-c-major-bwv-1009/#recordings)

## Convert to WAV files

These files can likely be converted on command line using `ffmpeg`, but I converted them using VLC for this example.  I have included some screenshots of the main steps.

1. File --> Convert / Save.  Add file(s) to convert

<img src="resources/vlc-open-media.png" width="600"/>

2. Create a new profile called "wav" with the settings shown below


<img src="resources/vlc-convert.png" width="600"/>
<img src="resources/vlc-wav-profile-encapsulation.png" width="600"/>
<img src="resources/vlc-wav-profile-audio-codec.png" width="600"/>

3. Save profile and click Convert!

## Clip WAV files

In [57]:
import importlib
import wav_utils
importlib.reload(wav_utils)
from wav_utils import load_wav_file, clip_wav_arr, convert_frames_to_seconds, save_wave_arr, mix_wav_arrs

input_wav_file_1 = "downloaded/Cello Suite no. 3 in C, BWV 1009 - 3. Courante-converted.wav"
output_wav_file_1 = "output/individual-clip-01.wav"

input_wav_file_2 = "downloaded/The Carnival of the Animals - XIII. The Swan (Solo piano version)-converted.wav"
output_wav_file_2 = "output/individual-clip-02.wav"

Load WAV files

In [32]:
signal_1, frame_rate_1 = load_wav_file(input_wav_file_1)
signal_2, frame_rate_2 = load_wav_file(input_wav_file_2)

print(f"{input_wav_file_1}")
print(f"\tframe_rate:     {frame_rate_1}")
print(f"\tnum_frames:     {len(signal_1)}")
print(f"\tlength_seconds: {convert_frames_to_seconds(len(signal_1)):.1f}")

print(f"{input_wav_file_2}")
print(f"\tframe_rate:     {frame_rate_2}")
print(f"\tnum_frames:     {len(signal_2)}")
print(f"\tlength_seconds: {convert_frames_to_seconds(len(signal_2)):.1f}")

downloaded/Cello Suite no. 3 in C, BWV 1009 - 3. Courante-converted.wav
	frame_rate:     44100
	num_frames:     5288879
	length_seconds: 119.9
downloaded/The Carnival of the Animals - XIII. The Swan (Solo piano version)-converted.wav
	frame_rate:     44100
	num_frames:     8796212
	length_seconds: 199.5


Create 20 second clips to mix

In [34]:
# NOTE: the second wav file is very quiet the first 7 seconds, so start clip later
clipped_signal_1 = clip_wav_arr(signal_1, start=0, end=20, unit="second")
clipped_signal_2 = clip_wav_arr(signal_2, start=7, end=27, unit="second")

# write them to file
print(output_wav_file_1)
save_wave_arr(clipped_signal_1, output_wav_file_1)

print(output_wav_file_2)
save_wave_arr(clipped_signal_2, output_wav_file_2)

output/individual-clip-01.wav
output/individual-clip-02.wav


## Create mixed WAV files

The goal is to simulate having two microphones in a room where both the cello and piano music is being played.

- **Mix 1**: mimic cello music being closer and thus a little louder
- **Mix 2**: mimic piano music being closer and thus a little louder

In [67]:
indiv_input_file_1 = "output/individual-clip-01.wav" # cello
indiv_input_file_2 = "output/individual-clip-02.wav" # piano
mix_output_file_1 = "output/mix-clip-01.wav" # cello louder than piano
mix_output_file_2 = "output/mix-clip-02.wav" # piano louder than cello
frac_volume = 0.7 # fraction of volume in mix allocated to the "primary" music

# load input files
signal_1, frame_rate_1 = load_wav_file(indiv_input_file_1)
signal_2, frame_rate_2 = load_wav_file(indiv_input_file_2)

# create the mixes
mix_1 = mix_wav_arrs([signal_1, signal_2], [frac_volume, 1.0 - frac_volume])
mix_2 = mix_wav_arrs([signal_1, signal_2], [1.0 - frac_volume, frac_volume])

# save mixes to file
print(mix_output_file_1)
save_wave_arr(mix_1, mix_output_file_1)

print(mix_output_file_2)
save_wave_arr(mix_2, mix_output_file_2)

output/mix-clip-01.wav
output/mix-clip-02.wav
